# [Message queue](https://en.wikipedia.org/wiki/Message_queue)

![](https://i0.wp.com/robtiffany.com/wp-content/uploads/2009/10/message_queue.png?fit=500%2C194)

* __Asynchronous communications protocol__, meaning that the sender and receiver of the message do not need to interact with the message queue at the same time.
    * Update part of a web page with more relevant information
    * An application may need to notify another that an event has occurred, but does not need to wait for a response.
    * In publish/subscribe systems, an application "publishes" information for any number of clients to read.

* Three standards have emerged which are used in open source message queue implementations:
    * Advanced Message Queuing Protocol (AMQP) – feature-rich message queue protocol, approved as ISO/IEC 19464 since April 2014
    * Streaming Text Oriented Messaging Protocol (STOMP) – simple, text-oriented message protocol
    * MQTT (formerly MQ Telemetry Transport) - lightweight message queue protocol especially for embedded devices
    


## A number of technologies

* IBM MQ. 
* RabbitMQ. 
* ActiveMQ. 
* Apache Kafka. 
* Aurea CX Messenger. 
* Solace PubSub+
* Anypoint MQ.
* Red Hat AMQ.

# AMQP

# [RabbitMQ](https://www.rabbitmq.com/)

## [Rabbimq su docker](https://hub.docker.com/_/rabbitmq/)

## [Tutorials code](https://github.com/rabbitmq/rabbitmq-tutorials)

```
docker run -d --hostname my-rabbit -p 5671:5671 -p 5672:5672 -p 15672:15672 -p 25672:25672 --name some-rabbit rabbitmq:3
```
```
docker ps
sudo docker exec -i -t fac0e279e9a4 /bin/bash
rabbitmqctl list_queues
```

In [1]:
!docker run -d --hostname my-rabbit -p 5671:5671 -p 5672:5672 -p 15672:15672 -p 25672:25672 --name some-rabbit rabbitmq:3

docker: Error response from daemon: Conflict. The container name "/some-rabbit" is already in use by container "bb6b0d437c59d86e4ea2d643c69dd042b880de40d7bbf65cd702b8f52faaaf10". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [6]:
!docker ps -a | grep rabbit

bb6b0d437c59        rabbitmq:3                          "docker-entrypoint..."   6 days ago          Up 46 hours                 0.0.0.0:5671-5672->5671-5672/tcp, 0.0.0.0:15672->15672/tcp, 4369/tcp, 0.0.0.0:25672->25672/tcp   some-rabbit


In [6]:
!docker start bb6b0d437c59

bb6b0d437c59


# Nodejs code for RabbitMQ tutorials

Here you can find Python code examples from [RabbitMQ tutorials](http://www.rabbitmq.com/getstarted.html).

To successfully use the examples you will need a running RabbitMQ broker (see sectino above on how to run it on docker)


## Nodejs tutorial (i.e. javascript)


[Tutorial one: "Hello World!"](http://www.rabbitmq.com/tutorial-one-javascript.html): The simplest thing that does something

![](http://www.rabbitmq.com/img/tutorials/python-one.png)


[Tutorial two: Work Queues](http://www.rabbitmq.com/tutorial-two-javascript.html): Distributing (time-consuming) tasks among workers

![](http://www.rabbitmq.com/img/tutorials/python-two.png)


[Tutorial three: Publish/Subscribe](http://www.rabbitmq.com/tutorial-three-javascript.html): Sending messages to many consumers at once

![](http://www.rabbitmq.com/img/tutorials/exchanges.png)



[Tutorial four: Routing](http://www.rabbitmq.com/tutorial-four-javascript.html): Receiving messages selectively

![](http://www.rabbitmq.com/img/tutorials/direct-exchange.png)


[Tutorial five: Topics](http://www.rabbitmq.com/tutorial-five-javascript.html): Receiving messages based on a pattern (topics)

![](http://www.rabbitmq.com/img/tutorials/python-five.png)


[Tutorial six: RPC](http://www.rabbitmq.com/tutorial-six-javascript.html): Request/reply pattern example

![](http://www.rabbitmq.com/img/tutorials/python-six.png)


## Note: the code running on a jupyter cell hangs. This is probably due to the interaction of Tornado (the Web Server underlying Jupyter) with the async Rabbitmq

## For this reason, It is better running this notebook on shells

In [7]:
!npm install amqplib

npm WARN saveError ENOENT: no such file or directory, open '/home/andrea/Documents/University/Didattica/RC2018/notebooks/AMQP/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/home/andrea/Documents/University/Didattica/RC2018/notebooks/AMQP/package.json'
npm WARN AMQP No description
npm WARN AMQP No repository field.
npm WARN AMQP No README data
npm WARN AMQP No license field.

+ amqplib@0.5.290m......] | postinstall: WARN AMQP No license field.
updated 1 package and audited 25 packages in 2.929s
found 0 vulnerabilities



![](http://www.rabbitmq.com/img/tutorials/sending.png)

In [ ]:
# %load code/send.js
#!/usr/bin/env node

var amqp = require('amqplib/callback_api');

// connect to RabbitMQ server
amqp.connect('amqp://localhost', function(err, conn) {
// create a queue. It is idempotent. You do not know
// whether will first the send or the receive  access the broker    conn.createChannel(function(err, ch) {
// and the queue must exist
    var q = 'hello';
    var msg = 'Hello World!';
// create a queue. It is idempotent
    ch.assertQueue(q, {durable: false});
    ch.sendToQueue(q, Buffer.from(msg));
    console.log(" [x] Sent %s", msg);
  });
// clode the connection and exit
  setTimeout(function() { conn.close(); process.exit(0) }, 500);
});


In [2]:
%%bash
terminator -T 'SEND' --command='code/send.js' --layout=class


![](http://www.rabbitmq.com/img/tutorials/receiving.png)

In [31]:
%%writefile code/receive.js
#!/usr/bin/env node

var amqp = require('amqplib/callback_api');

amqp.connect('amqp://localhost', function(err, conn) {
  conn.createChannel(function(err, ch) {
    var q = 'hello';
// create a queue. It is idempotent. You do not know
// whether will first the send or the receive  access the broker
// and the queue must exist    
      ch.assertQueue(q, {durable: false});
    console.log(" [*] Waiting for messages in %s. To exit press CTRL+C", q);
    ch.consume(q, function(msg) {
      console.log(" [x] Received %s", msg.content.toString());
    }, {noAck: true});
  });
  // trick to make it working for a single call ... not really useful!
  //setTimeout(function() { conn.close(); process.exit(0) }, 500);
});


Overwriting code/receive.js


In [4]:
%%bash
terminator -T 'RECEIVE' --command='code/receive.js' --profile=class

In [10]:
!docker ps -a | grep rabbit

bb6b0d437c59        rabbitmq:3                          "docker-entrypoint..."   6 days ago          Up 2 days                   0.0.0.0:5671-5672->5671-5672/tcp, 0.0.0.0:15672->15672/tcp, 4369/tcp, 0.0.0.0:25672->25672/tcp   some-rabbit


```bash
sudo docker exec -i -t bb6b0d437c59 /bin/bash #by ID

sudo rabbitmqctl list_queues
```

![](http://www.rabbitmq.com/img/tutorials/python-two.png)

# MQTT

# https://github.com/mqtt/mqtt.github.io/wiki

# https://www.hivemq.com/mqtt-essentials/

![](https://www.hivemq.com/wp-content/uploads/Screen-Shot-2014-10-22-at-12.21.07.png)

# https://www.hivemq.com/blog/mqtt-essentials-part-5-mqtt-topics-best-practices

![image.png](https://vasters.com/images/content/mqtt.png)



# http://www.steves-internet-guide.com/understanding-mqtt-qos-levels-part-1/


## QOS 0: At most once (fire and forget)
 
## QOS 1: At least once 
![](http://www.steves-internet-guide.com/wp-content/uploads/mqtt-publish-flow-qos-1.jpg)
## QOS 1: Exactly once 
![](http://www.steves-internet-guide.com/wp-content/uploads/mqtt-publish-flow-qos-2.jpg)


In [2]:
%%bash
terminator -T 'RECEIVE' --command='service mosquitto stop' --profile=class

In [9]:
%%bash
terminator -T 'RECEIVE' --command='docker run -it -p 1883:1883 -p 9001:9001 eclipse-mosquitto' --profile=class


http://www.steves-internet-guide.com/mosquitto_pub-sub-clients/

In [10]:
%%bash
terminator -T 'RECEIVE' --command='mosquitto_sub -h 127.0.0.1 -p 1883 -t house/#' --profile=class

In [11]:
!mosquitto_pub -h 127.0.0.1 -p 1883 -m {\"status\":\"ON\"} -t house/roomo1/light1 -d

Client mosqpub|22996-andrea-La sending CONNECT
Client mosqpub|22996-andrea-La received CONNACK
Client mosqpub|22996-andrea-La sending PUBLISH (d0, q0, r0, m1, 'house/roomo1/light1', ... (15 bytes))
Client mosqpub|22996-andrea-La sending DISCONNECT


## http://vasters.com/blog/From-MQTT-to-AMQP-and-back/#null

![](http://vasters.com/images/content/federation.png)